## PoC dos conceitos utilizando o Redis

Varre todos os arquivos JSON gerando uma lista com os dados pertinentes a irem para o Redis

In [1]:
import boto3

SESSION = boto3.session.Session(region_name='us-east-1', profile_name='sofie')
DYNAMODB = SESSION.resource('dynamodb')
TABLE = DYNAMODB.Table('table_micro_task_in_person')

FINAL_LIST = list()

scan_params = dict(
    FilterExpression='#status.#state = :n',
    ExpressionAttributeNames={'#state': 'state', '#status': 'status'},
    ExpressionAttributeValues={':n': 'NEW'},
)

while True:
    response = TABLE.scan(**scan_params)
    
    for data in response.get('Items'):
        item = {
            'category': data['task']['category'],
            'type': data['task']['type'],
            'task_id': data['task_id'],
            'reward': float(data['task']['reward']),
            'name': data['sofie_place']['name'],
            'address': data['address']['formatted_address'],
            'lat': float(data['google_maps']['results'][0]['geometry']['location']['lat']),
            'lng': float(data['google_maps']['results'][0]['geometry']['location']['lng']),
            'task_name': data['task']['name']
        }
        FINAL_LIST.append(item)
    
    last_key = response.get('LastEvaluatedKey')
    if not last_key:
        break
        
    scan_params['ExclusiveStartKey'] = last_key
    
print(f'{len(FINAL_LIST)} itens na lista')


865 itens na lista


In [2]:
FINAL_LIST[0]

{'category': 'in_person',
 'type': 'inspection',
 'task_id': 'f54ea170-91f4-40b8-8959-fa47cdb372c8',
 'reward': 10.0,
 'name': 'EMPORIO SHOW ARTE E CAFE',
 'address': 'Rua Dimitri Sensaud de Lavoud, 40 - Vila Campesina, Osasco',
 'lat': -23.5427299,
 'lng': -46.7735081,
 'task_name': 'PAT'}

## HIGIENIZAÇÃO DO REDIS

Limpa todas as entradas do Redis referentes ao assunto MICROTASK recriando-as com as informações atualizadas

In [3]:
from redis import Redis

# conn = Redis('localhost', 6379)
conn = Redis('172.30.1.20', 5052)

In [4]:
#raise Exception('Cuidado!')

for each in conn.keys('SOFIE:MICROTASK:*#'):
    conn.delete(each)

for each in FINAL_LIST:
    conn.hmset(f'SOFIE:MICROTASK:{each["task_id"]}:DATA#', each)
    conn.geoadd('SOFIE:MICROTASK:IN_PERSON#', each['lng'], each['lat'], each['task_id'])
               
print(f'*** PROCESSADO COM SUCESSO -> {len(conn.keys('SOFIE:MICROTASK:*#'))} ITENS ***')

*** PROCESSADO COM SUCESSO -> 12880 ITENS ***


# LISTAGEM DE TAREFAS

In [25]:
import datetime

for each in conn.keys('SOFIE:MICROTASK:*:RESERVED:*#'):
    print(each)
    ttl = conn.ttl(each)
    conn.setex(each, ttl, datetime.datetime.utcnow().isoformat())
    

b'SOFIE:MICROTASK:9abec9ef-1010-4b8f-99ee-a6448c716b5a:RESERVED:sonia@mysofie.com#'
b'SOFIE:MICROTASK:93f91aa0-eb46-4c29-bda0-2953acfc84e0:RESERVED:sonia@mysofie.com#'
b'SOFIE:MICROTASK:2fd10e54-d111-41aa-a6b3-0923e9bfe6df:RESERVED:pritybenites@gmail.com#'
b'SOFIE:MICROTASK:6e8cb5ec-525a-461e-af6b-e1dfe766e02e:RESERVED:sonia@mysofie.com#'
b'SOFIE:MICROTASK:ef5a837c-4a6c-4d2b-a1de-123cd26ed369:RESERVED:sonia@mysofie.com#'
b'SOFIE:MICROTASK:56242317-33a0-4be9-b809-5695c49e7769:RESERVED:thiago.filadelfo@gmail.com#'
b'SOFIE:MICROTASK:0db15e17-7899-4369-b752-b6c53ac85b22:RESERVED:sonia@mysofie.com#'
b'SOFIE:MICROTASK:ac350204-4ce1-4eb7-82b2-abc0c9c64df2:RESERVED:sonia@mysofie.com#'
b'SOFIE:MICROTASK:5de116ce-2227-425f-a3ff-a834955f1632:RESERVED:sonia@mysofie.com#'
b'SOFIE:MICROTASK:47aeb48a-4235-4fb0-b1c1-57a20c7f6251:RESERVED:sonia@mysofie.com#'
b'SOFIE:MICROTASK:ce00bbb1-349f-4b84-bea8-956770fdadd1:RESERVED:mario@arrayof.io#'
b'SOFIE:MICROTASK:6fcdb4e0-05ed-4c64-a911-298956e4f90e:RESERVED

In [24]:
conn.ttl('SOFIE:MICROTASK:9abec9ef-1010-4b8f-99ee-a6448c716b5a:RESERVED:sonia@mysofie.com#')

9966